In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
fields = ["Date_HPSC", "TotalLabs", "Positive", "Test24", "PRate"]
cov_nums = pd.read_csv("LaboratoryLocalTimeSeriesHistoricView.csv", usecols=fields)
cov_nums.head()

,Date_HPSC,TotalLabs,Positive,PRate,Test24
0,2020/03/18 11:00:00+00,6457,395,6.1,0
1,2020/03/19 11:00:00+00,7403,465,6.3,946
2,2020/03/20 11:00:00+00,8872,570,6.4,1469
3,2020/03/21 11:00:00+00,10436,625,6.0,1564
4,2020/03/22 11:00:00+00,11414,744,6.5,978


In [3]:
def positive(row):
    pos = (row['Test24'] / 100) * row['PRate']
    return round(pos)

cov_nums['daily_pos'] = cov_nums.apply(positive, axis=1)
cov_nums = cov_nums.iloc[1:]
cov_nums.tail()

,Date_HPSC,TotalLabs,Positive,PRate,Test24,daily_pos
389,2021/04/11 11:00:00+00,4184407,248665,5.9,14355,847
390,2021/04/12 11:00:00+00,4196325,248992,5.9,11918,703
391,2021/04/13 11:00:00+00,4209975,249344,5.9,13650,805
392,2021/04/14 11:00:00+00,4226847,249797,5.9,16872,995
393,2021/04/15 11:00:00+00,4241709,250215,5.9,14862,877


In [4]:
test_info = ["Date_HPSC", "Test24"]
test_df = cov_nums[test_info]
test_df = pd.DataFrame(test_df)
test_df['Date_HPSC'] = pd.to_datetime(test_df['Date_HPSC']).dt.date
test_df.head()

,Date_HPSC,Test24
1,2020-03-19,946
2,2020-03-20,1469
3,2020-03-21,1564
4,2020-03-22,978
5,2020-03-23,2581


In [5]:
pos = ['Date_HPSC','Test24','daily_pos','PRate']
pos_df = cov_nums[pos]
pos_df = pd.DataFrame(pos_df)
pos_df['Date_HPSC'] = pd.to_datetime(pos_df['Date_HPSC']).dt.date

In [6]:
def seq(row):
    length = (int(row['Test24']) - int(row['daily_pos']))
    daily_pos = int(row['daily_pos'])
    init_seq = [0] * length
    init_pos = [1] * daily_pos
    seq_list = init_seq + init_pos
    for i in range(len(seq_list)-1, 0, -1):
        j = random.randint(0, i+1)
        seq_list[i], seq_list[j] = seq_list[j], seq_list[i]
    return list(seq_list)

pos_df['seq_list'] = pos_df.apply(seq, axis=1)
pos_df = pd.DataFrame(pos_df)


In [7]:
def hasPositive(block):
    for node in block:
        if node == 1:
            return True
    return False

def splitBlock(block):
    halfBlock = len(block) // 2
    return block[:halfBlock], block[halfBlock:]

def binSplit(data, count):
    count += 1
    #print("Data:", data)
    if len(data) > 1 and hasPositive(data):     # Branch is complete
        left, right = splitBlock(data)      # Split the block
       # print("Left:", left, "\tRight:", right)
        if len(left) <= 1 or not hasPositive(left):     # Left branch is complete
        #    print("Left branch is complete")
            return binSplit(right, count)
        result, count = binSplit(left, count)
      #  print("Result:", result)
        if result is True:
       #     print("Result was True")
            return binSplit(right, count)
      #  print("Adding result to right and splitting")
        return binSplit((result + right), count)
    else:
     #   print("Returning True")
        return True, count

def runtime(row):
    answer = binSplit(row, 0)[1]
    return answer

pos_df['pool_test'] = pos_df['seq_list'].apply(runtime)
pos_df['pool_test'].head()
pos_df = pos_df.drop(['seq_list'], axis=1)

df = pos_df
df.head()

,Date_HPSC,Test24,daily_pos,PRate,pool_test
1,2020-03-19,946,60,6.3,358
2,2020-03-20,1469,94,6.4,595
3,2020-03-21,1564,94,6.0,585
4,2020-03-22,978,64,6.5,408
5,2020-03-23,2581,157,6.1,982
